In [1]:
library(verification)
library(caret)
library(e1071) # SVM
library(gbm) # GBM

Warning message:
"package 'verification' was built under R version 3.6.2"Loading required package: fields
Warning message:
"package 'fields' was built under R version 3.6.2"Loading required package: spam
Warning message:
"package 'spam' was built under R version 3.6.2"Loading required package: dotCall64
Warning message:
"package 'dotCall64' was built under R version 3.6.2"Loading required package: grid
Spam version 2.5-1 (2019-12-12) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: 'spam'

The following objects are masked from 'package:base':

    backsolve, forwardsolve

Loading required package: maps
Warning message:
"package 'maps' was built under R version 3.6.2"See https://github.com/NCAR/Fields for
 an extensive vignette, other supplements and source code 
Loading required package: boot

In [2]:
hotel = read.csv("../Data/hotel_bookings.csv")

In [16]:
kappa=function(x){
      n=sum(x)
      pobs=(x[1,1]+x[2,2])/n
      pexp=(sum(x[1,])*sum(x[,1])+sum(x[2,])*sum(x[,2]))/n^2
      kappa=(pobs-pexp)/(1-pexp)
      t1=0
      t2=0
      t3=0
      pii=x/n
      pidot=apply(pii,1,sum)
      pdotj=apply(pii,2,sum)
      for(i in 1:2){
            t1 = t1 + pii[i,i]*((1-pexp) - (1-pobs)*(pidot[i]+pdotj[i]))^2
      }
      t2 = pii[1,2]*(pdotj[1]+pidot[2])^2 + pii[2,1]*(pdotj[2] + pidot[1])^2
      t3 = (pobs*pexp-2*pexp+pobs)^2
      vhat = (t1 + t2*(1-pobs)^2 -t3)/(n*(1-pexp)^4)
      se=sqrt(vhat)
      return(c(kappa,se))
}


class.sum=function(truth,predicted){
     xt=table(truth,round(predicted+0.000001))
     pcc=round(100*sum(diag(xt))/sum(xt),2)
     spec=round(100*xt[1,1]/sum(xt[1,]),2)
     sens=round(100*xt[2,2]/sum(xt[2,]),2)
     kap=round(kappa(xt)[1],4)
     au=round(roc.area(truth,predicted)$A,4)
     return(cbind(c("Percent Correctly Classified = ","Specificity = ","Sensitivity = ","Kappa =","AUC= "),c(pcc,spec,sens,kap,au)))
     }

In [3]:
head(hotel)

hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NULL,NULL,0,Transient,0,0,0,Check-Out,2015-07-01
Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NULL,NULL,0,Transient,0,0,0,Check-Out,2015-07-01
Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NULL,NULL,0,Transient,75,0,0,Check-Out,2015-07-02
Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304,NULL,0,Transient,75,0,0,Check-Out,2015-07-02
Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240,NULL,0,Transient,98,0,1,Check-Out,2015-07-03
Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240,NULL,0,Transient,98,0,1,Check-Out,2015-07-03


In [6]:
unique(hotel$is_canceled)

[1] 0 1

In [4]:
summary(hotel)

          hotel        is_canceled       lead_time   arrival_date_year
 City Hotel  :79330   Min.   :0.0000   Min.   :  0   Min.   :2015     
 Resort Hotel:40060   1st Qu.:0.0000   1st Qu.: 18   1st Qu.:2016     
                      Median :0.0000   Median : 69   Median :2016     
                      Mean   :0.3704   Mean   :104   Mean   :2016     
                      3rd Qu.:1.0000   3rd Qu.:160   3rd Qu.:2017     
                      Max.   :1.0000   Max.   :737   Max.   :2017     
                                                                      
 arrival_date_month arrival_date_week_number arrival_date_day_of_month
 August :13877      Min.   : 1.00            Min.   : 1.0             
 July   :12661      1st Qu.:16.00            1st Qu.: 8.0             
 May    :11791      Median :28.00            Median :16.0             
 October:11160      Mean   :27.17            Mean   :15.8             
 April  :11089      3rd Qu.:38.00            3rd Qu.:23.0             
 June 

In [4]:
str(hotel)

'data.frame':	119390 obs. of  32 variables:
 $ hotel                         : Factor w/ 2 levels "City Hotel","Resort Hotel": 2 2 2 2 2 2 2 2 2 2 ...
 $ is_canceled                   : int  0 0 0 0 0 0 0 0 1 1 ...
 $ lead_time                     : int  342 737 7 13 14 14 0 9 85 75 ...
 $ arrival_date_year             : int  2015 2015 2015 2015 2015 2015 2015 2015 2015 2015 ...
 $ arrival_date_month            : Factor w/ 12 levels "April","August",..: 6 6 6 6 6 6 6 6 6 6 ...
 $ arrival_date_week_number      : int  27 27 27 27 27 27 27 27 27 27 ...
 $ arrival_date_day_of_month     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ stays_in_weekend_nights       : int  0 0 0 0 0 0 0 0 0 0 ...
 $ stays_in_week_nights          : int  0 0 1 1 2 2 2 2 3 3 ...
 $ adults                        : int  2 2 1 1 2 2 2 2 2 2 ...
 $ children                      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ babies                        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ meal                          : Factor w/ 5 levels "BB","

In [18]:
hotel_samp = hotel[sample(nrow(hotel), 4000), ]

In [14]:
str(hotel_samp)

'data.frame':	200 obs. of  32 variables:
 $ hotel                         : Factor w/ 2 levels "City Hotel","Resort Hotel": 2 2 1 2 2 2 1 1 1 1 ...
 $ is_canceled                   : int  1 1 1 1 0 1 0 1 0 0 ...
 $ lead_time                     : int  130 181 28 58 2 227 89 195 0 133 ...
 $ arrival_date_year             : int  2016 2017 2016 2016 2017 2017 2015 2015 2017 2016 ...
 $ arrival_date_month            : Factor w/ 12 levels "April","August",..: 11 8 3 8 6 9 10 12 4 8 ...
 $ arrival_date_week_number      : int  43 9 51 13 28 18 48 38 6 11 ...
 $ arrival_date_day_of_month     : int  21 1 15 26 15 4 28 17 6 11 ...
 $ stays_in_weekend_nights       : int  0 6 1 2 2 2 2 0 1 1 ...
 $ stays_in_week_nights          : int  1 19 3 1 1 8 1 2 1 2 ...
 $ adults                        : int  2 2 3 2 2 2 2 2 2 2 ...
 $ children                      : int  2 0 0 2 0 0 0 0 0 0 ...
 $ babies                        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ meal                          : Factor w/ 5 lev

# Support Vector Machines

## Untuned SVM

In [21]:
set.seed(424)
data = hotel_samp
# Nests = subset(nest, select = c(Nest, NumTree3to6in, NumTree9to15in,
#                                 NumTree1to3in, NumTree6to9in, NumDownSnags,
#                                 NumConifer, NumTreegt15in, NumSnags, 
#                                 NumTreelt1in, StandType, PctShrubCover))


hotel.svm.xvalpred=rep(0,nrow(data))
xvs=rep(1:10,length=nrow(data))
xvs=sample(xvs)
for(i in 1:10){
      train=data[xvs!=i,]
      test=data[xvs==i,]
      glub=svm(as.factor(is_canceled)~ . - reservation_status,probability=TRUE,data=train)
      hotel.svm.xvalpred[xvs==i]=attr(predict(glub,test,probability=TRUE),"probabilities")[,2]
}

table(data$is_canceled,round(hotel.svm.xvalpred))
class.sum(data$is_canceled,hotel.svm.xvalpred)

   
       0    1
  0  289 2230
  1  930  551

Percent Correctly Classified =,21
Specificity =,11.47
Sensitivity =,37.2
Kappa =,-0.4346
AUC=,0.1482


## Tuned SVM

# Gradient Boosting Machines

## Untuned Gradient Boosting Machines

In [22]:
set.seed(424)
hotel.gbm.xvalpr=rep(0,nrow(data))
xvs=rep(1:10,length=nrow(data))
xvs=sample(xvs)
for(i in 1:10){
      train=data[xvs!=i,]
      test=data[xvs==i,]
      glub=gbm(is_canceled~ . - reservation_status,distribution="bernoulli",n.trees=5000,data=train)
      hotel.gbm.xvalpr[xvs==i]=predict(glub,newdata=test,type="response",n.trees=5000)
}

table(data$is_canceled,round(hotel.gbm.xvalpr))
class.sum(data$is_canceled,hotel.gbm.xvalpr)

   
       0    1
  0 1903  616
  1  741  740

Percent Correctly Classified =,66.08
Specificity =,75.55
Sensitivity =,49.97
Kappa =,0.2596
AUC=,0.6568


## Tuned Gradient Boosting Machines

In [23]:
set.seed(732)
fitControl = trainControl(method = "cv", number = 10 )

In [ ]:
gbmGrid = expand.grid(interaction.depth = c(12, 14, 16, 18, 20), n.trees = c(25,50,75,100), shrinkage = c(0.01, 0.05, 0.1, 0.2 ), n.minobsinnode=10)
gbmFit = train( as.factor(is_canceled)~ . - reservation_status , method="gbm", tuneGrid = gbmGrid, trControl = fitControl, data=data)
gbmFit

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 27: countryAIA has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 28: countryALB has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 29: countryAND has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 32: countryARM has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 33: countryASM has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 34: countryATA has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 35: countryATF has no variation."Warning message in (function (x, y, offse